In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

# a = torch.arange(15).reshape(3, 5).float()
# print(a)
# vals, inds = a.topk(k=4, dim=-1)
# print(vals)
# print(inds)
# print(vals.shape)

tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.]])
tensor([[ 4.,  3.,  2.,  1.],
        [ 9.,  8.,  7.,  6.],
        [14., 13., 12., 11.]])
tensor([[4, 3, 2, 1],
        [4, 3, 2, 1],
        [4, 3, 2, 1]])
torch.Size([3, 4])


In [2]:
# !gdown --fuzzy "https://drive.google.com/file/d/12ycYSzLIG253AFN35Y6qoyf9wtkOjakp/view?usp=sharing"

In [3]:
# !tar -zxvf "./2017-01-trnmted.tgz"
# !tar -zxvf "./2017-01-trnmted/texts/DeEnItNlRo/DeEnItNlRo/DeEnItNlRo-DeEnItNlRo.tgz"
# !mv ./DeEnItNlRo-DeEnItNlRo ./texts
# !mkdir ./clean_texts
# !mkdir ./tokenizers
# !rm "./2017-01-trnmted.tgz"
# !rm -rf "./2017-01-trnmted"

In [2]:
import data
from pathlib import Path

# data.convert_files(Path('./texts'), Path('./clean_texts'))
# data.train_tokenizers(Path('./clean_texts'), Path('./tokenizers'))

In [3]:
from train_model import train_model, train_epoch, translate_test_set

# train_model("./clean_texts", "./tokenizers", 5)

In [4]:
from pathlib import Path

import torch
from sacrebleu.metrics import BLEU
from tokenizers import Tokenizer
from tqdm.auto import trange, tqdm

from data import TranslationDataset
from decoding import translate, get_attn_mask
from model import TranslationModel

import wandb

In [5]:
data_dir = Path("./clean_texts")
tokenizer_path = Path("./tokenizers")
src_tokenizer = Tokenizer.from_file(str(tokenizer_path / "tokenizer_de.json"))
tgt_tokenizer = Tokenizer.from_file(str(tokenizer_path / "tokenizer_en.json"))

In [9]:
config = {
    "batch_size" : 6,
    "lr" : 3e-4,
    "max_len" : 128,  # might be enough at first
    "num_encoder_layers" : 3,
    "num_decoder_layers" : 3,
    "emb_size" : 256,
    "dim_feedforward" : 512,
    "n_head" : 8,
    "dropout_prob" : 0.1,
}


# train_dataset = TranslationDataset(
#     data_dir / "train.de.txt",
#     data_dir / "train.en.txt",
#     src_tokenizer,
#     tgt_tokenizer,
#     max_len=128 #config["max_len"],
# )

# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=6,
#     collate_fn = train_dataset.collate_translation_data,
#     # shuffle=True,
# )

In [7]:
val_dataset = TranslationDataset(
    data_dir / "val.de.txt",
    data_dir / "val.en.txt",
    src_tokenizer,
    tgt_tokenizer,
    max_len=config["max_len"],
)


val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    config["batch_size"],
    collate_fn = val_dataset.collate_translation_data,
)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

src_pad_id = tgt_tokenizer.token_to_id("[PAD]")
tgt_pad_id = tgt_tokenizer.token_to_id("[PAD]")

model = TranslationModel(
    config["num_encoder_layers"],
    config["num_decoder_layers"],
    config["emb_size"],
    config["dim_feedforward"],
    config["n_head"],
    src_tokenizer.get_vocab_size(),
    tgt_tokenizer.get_vocab_size(),
    config["dropout_prob"],
    src_pad_id,
    tgt_pad_id,
    config["max_len"]
)

# model.load_state_dict(torch.load("checkpoint_last.pth")["model_state_dict"])


print("Total no. of model parameters:",
    "pytorch_total_params =", sum(p.numel() for p in model.parameters())
)
model.to(device);

Total no. of model parameters: pytorch_total_params = 27024688


In [13]:
model.load_state_dict(torch.load("saved.pth")["model_state_dict"])

<All keys matched successfully>

In [9]:
# from train_model import train_epoch, evaluate, translate_test_set

# num_epochs=1
# optimizer = torch.optim.Adam(model.parameters(), config["lr"])
# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer,
#     config["lr"],
#     steps_per_epoch=len(train_dataloader),
#     epochs=num_epochs,
#     pct_start=0.1
# )
# CELoss = torch.nn.CrossEntropyLoss(ignore_index=tgt_pad_id)

# min_val_loss = float("inf")

# for epoch in trange(1, num_epochs + 1):
#     val_loss = evaluate(model, train_dataloader, CELoss, device, src_tokenizer, tgt_tokenizer)

In [10]:
# from train_model import evaluate

In [15]:

# wandb.init(config=config, project="DL-LHW2")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dlishudi. Use `wandb login --relogin` to force relogin


In [13]:
# for epoch in trange(1):
#     val_loss, table = evaluate(model, val_dataloader, CELoss, device, src_tokenizer, tgt_tokenizer, wandb)
#     print(val_loss)

  0%|          | 0/1 [00:00<?, ?it/s]

10.475229707924095


In [17]:
wandb.log({"examples" : table})

In [69]:
wandb.log({
    "epoch" : epoch,
    "val_loss" : val_loss,
    "first batch translation" : table
})

In [18]:
wandb.finish()

In [42]:
a = torch.arange(10* 5).reshape(10,5)
mask = torch.tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 1]).bool()
a[mask]

tensor([[ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [45, 46, 47, 48, 49]])

In [82]:
translate_test_set(model, data_dir, tokenizer_path, batch_size=8)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/dima/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_485/2840094636.py", line 1, in <module>
    translate_test_set(model, data_dir, tokenizer_path, batch_size=8)
  File "/home/dima/DL/LHW2/DL-LHW1/train_model.py", line 275, in translate_test_set
    greed_out = translate(model, src_sentences, src_tokenizer, tgt_tokenizer, "greedy", device)
  File "/home/dima/.local/lib/python3.10/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/home/dima/DL/LHW2/DL-LHW1/decoding.py", line 199, in translate
  File "/home/dima/DL/LHW2/DL-LHW1/decoding.py", line 62, in _greedy_decode
    new_tokens = model.decode_last(
  File "/home/dima/DL/LHW2/DL-LHW1/model.py", line 120, in decode_last
    decoded = self.transformer.decoder(
  File "/home/dima/.local/lib/python3.10/si

In [79]:
translate_test_set(model, data_dir, tokenizer_path, batch_size=1)

--------------------------------------------------
torch.Size([1, 28, 256])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1, 256])
torch.Size([1, 30000])
--------------------------------------------------
--------------------------------------------------
torch.Size([1, 28, 256])
torch.Size([1, 2])
torch.Size([2, 2])
torch.Size([1, 2, 256])
torch.Size([1, 30000])
--------------------------------------------------
--------------------------------------------------
torch.Size([1, 28, 256])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([1, 3, 256])
torch.Size([1, 30000])
--------------------------------------------------
--------------------------------------------------
torch.Size([1, 28, 256])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([1, 4, 256])
torch.Size([1, 30000])
--------------------------------------------------
--------------------------------------------------
torch.Size([1, 28, 256])
torch.Size([1, 5])
torch.Size([5, 5])
torch.Size([1, 5, 256])
torch.Size([1,

In [19]:
from sacremoses import MosesDetokenizer, MosesPunctNormalizer

In [46]:
# src_tokenizer.token_to_id("[PAD]")
tgt_tokenizer.token_to_id("[PAD]")

1

In [39]:
decoded.split()

['W', 'hat', 'are', 'you', 'do', 'ing', '?']

In [83]:
text = "Hi Madina"
src_tokenizer.encode(text).tokens

['[BOS]', 'Hi', 'Mad', 'ina', '[EOS]']

In [102]:

from tokenizers.processors import TemplateProcessing

src_tokenizer.post_processor = TemplateProcessing(
    single="[EOS] $A [BOS]",
    pair="[EOS] $A [UNK] $B:1 [BOS]:1",
    special_tokens=[
        ("[EOS]", src_tokenizer.token_to_id("[EOS]")),
        ("[BOS]", src_tokenizer.token_to_id("[BOS]")),
        ("[UNK]", src_tokenizer.token_to_id("[UNK]"))
    ],
)

In [69]:
batch_encs = tgt_tokenizer.encode_batch(["Hello, Marina", "Hi Dima!"])

In [71]:
for i in batch_encs:
    print(i.tokens)

['[BOS]', 'Hello', ',', 'Mar', 'ina', '[EOS]']
['[BOS]', 'Hi', 'D', 'ima', '!', '[EOS]']


In [142]:
decoded = tgt_tokenizer.decode_batch(torch.Tensor(batch_ids).long().tolist())

In [151]:
model

NameError: name 'model' is not defined

In [26]:
from sacremoses import MosesPunctNormalizer
mpn = MosesPunctNormalizer()
"It ' s fine!".replace(" '", "'").replace("' ", "'")

"It's fine!"

In [ ]:
detok.deto

In [28]:
from sacremoses import MosesDetokenizer, MosesPunctNormalizer

detok = MosesDetokenizer()
mpn = MosesPunctNormalizer()

text = "Hello, World!"
encoded = src_tokenizer.encode(text).ids
decoded = src_tokenizer.decode(encoded)
detokenized = detok.detokenize(decoded.split())
normalized = mpn.normalize(decoded)
print(decoded)
print(detokenized)
print(normalized)

Hell o , World !
Hell o, World!
Hell o , World !


In [91]:
doubled = [decoded, decoded]
detok.detokenize(doubled)

'W hat are you do ing? W hat are you do ing?'

In [20]:
seqs = src_tokenizer.decode_batch([train_dataset[7][0].tolist(), train_dataset[8][0].tolist()])
# seqs = "WHAT , ARE YOU ' LL DOING ?"
print(seqs)
print(MosesDetokenizer().detokenize(seqs))
print(MosesPunctNormalizer().normalize(seqs))

['Ich erzähle Ihnen mal eine Geschichte , dann verstehen Sie mich vielleicht besser .', 'Eine wahre Geschichte -- kein Wort daran ist erfunden .']
Ich erzähle Ihnen mal eine Geschichte, dann verstehen Sie mich vielleicht besser. Eine wahre Geschichte -- kein Wort daran ist erfunden.
['Ich erzähle Ihnen mal eine Geschichte , dann verstehen Sie mich vielleicht besser .', 'Eine wahre Geschichte -- kein Wort daran ist erfunden .']


In [21]:
MosesPunctNormalizer().normalize(seqs)

"['Ich erzähle Ihnen mal eine Geschichte , dann verstehen Sie mich vielleicht besser .', 'Eine wahre Geschichte -- kein Wort daran ist erfunden .']"

In [11]:
seqs[0]

NameError: name 'seqs' is not defined

In [12]:
len(train_dataset)

0

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/12882 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [65]:
# src

In [69]:
for i in train_dataloader:
    for line in i["tgt"]:
        print(line)
    print(i["tgt"])
    break

tensor([[    2,   698,   175,   215,   467,    15,  2040,    17,     3,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    2,   194,   159,    10,    82,  2691,    64,   589,  4596,   155,
           223,   147,  1869,   155,   451,   155,   197,  1899,  3318,    30,
            42,    10,    76,  2433,  5685,    17,     3,     1,     1,     1,
             1,     1,     1,     1],
        [    2,    42,   223,   426,  7699,   880,   325,   197,  3479,    15,
           163,    42,   358,   155,  2075,   209,   162,   175,   199,   147,
           516,  1936,  7305,   255,   246,    42,   336,   155,   399,   147,
           333,  1227,    17,     3],
        [    2,   194,    42,   399,   168,  1428,  4112,    15,  6879,   341,
            42,   442,   168,    17,     3,     1,     1,     1,     1,     1,
             1,  